## Kaggle – DataTops®
Tu profe ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a tu profe a mejorar ese modelo?

## Métrica: 
Error de raíz cuadrada media (RMSE) es la desviación estándar de los valores residuales (errores de predicción). Los valores residuales son una medida de la distancia de los puntos de datos de la línea de regresión; RMSE es una medida de cuál es el nivel de dispersión de estos valores residuales. En otras palabras, le indica el nivel de concentración de los datos en la línea de mejor ajuste.


$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$


## Librerías

In [1]:
import numpy as np
import pandas as pd
import toolbox_ML as tb
import re
from PIL import Image
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import urllib.request

## Datos

In [2]:
# Carga de datos
df = pd.read_csv("./data/train.csv", index_col="laptop_ID")
test = pd.read_csv("./data/test.csv", index_col="laptop_ID")

# Convertimos nombres de columnas a minúsculas
df.columns = df.columns.str.lower()
test.columns = test.columns.str.lower()

## Exploración de los datos

In [3]:
display(df)
print(df.info())

,company,product,typename,inches,screenresolution,cpu,ram,memory,gpu,opsys,weight,price_in_euros
laptop_ID,,,,,,,,,,,,
755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00
...,...,...,...,...,...,...,...,...,...,...,...,...
28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00


<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   company           912 non-null    object 
 1   product           912 non-null    object 
 2   typename          912 non-null    object 
 3   inches            912 non-null    float64
 4   screenresolution  912 non-null    object 
 5   cpu               912 non-null    object 
 6   ram               912 non-null    object 
 7   memory            912 non-null    object 
 8   gpu               912 non-null    object 
 9   opsys             912 non-null    object 
 10  weight            912 non-null    object 
 11  price_in_euros    912 non-null    float64
dtypes: float64(2), object(10)
memory usage: 92.6+ KB
None


In [4]:
tb.describe_df(df)

,company,product,typename,inches,screenresolution,cpu,ram,memory,gpu,opsys,weight,price_in_euros
DATA_TYPE,object,object,object,float64,object,object,object,object,object,object,object,float64
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,19,480,6,17,36,107,9,37,93,9,165,603
CARDIN (%),2.08,52.63,0.66,1.86,3.95,11.73,0.99,4.06,10.2,0.99,18.09,66.12


In [5]:
tb.tipifica_variables(df)

,nombre_variable,tipo_sugerido
0,company,Numerica Discreta
1,product,Numerica Continua
2,typename,Categorica
3,inches,Numerica Discreta
4,screenresolution,Numerica Discreta
5,cpu,Numerica Discreta
6,ram,Categorica
7,memory,Numerica Discreta
8,gpu,Numerica Discreta
9,opsys,Categorica


## Procesado de datos

Nuestro target es la columna `Price_in_euros`

In [6]:
# Convertimos algunas a numéricas
df["ram_in_gb"] = df["ram"].str.replace("GB", "").astype(float)
test["ram_in_gb"] = test["ram"].str.replace("GB", "").astype(float)

df["weight_in_kg"] = df["weight"].str.replace("kg", "").astype(float)
test["weight_in_kg"] = test["weight"].str.replace("kg", "").astype(float)

df["res_y"] = df["screenresolution"].str.extract(r"x(\d+)", expand=False).astype(float)
test["res_y"] = test["screenresolution"].str.extract(r"x(\d+)", expand=False).astype(float)

In [7]:
def parse_storage(text):
    # Expresiones para detectar cada tipo y cantidad
    total = 0
    ssd = 0

    # Buscar todas las unidades tipo "256GB SSD", "1TB HDD", etc.
    matches = re.findall(r'(\d+\.?\d*)TB|\d+\.?\d*GB', text.upper())

    for part in text.split('+'):
        part = part.strip().upper()
        if 'SSD' in part or 'FLASH' in part:
            if 'TB' in part:
                ssd += float(re.search(r'(\d+\.?\d*)TB', part).group(1)) * 1024
            elif 'GB' in part:
                ssd += float(re.search(r'(\d+\.?\d*)GB', part).group(1))
        if 'TB' in part:
            total += float(re.search(r'(\d+\.?\d*)TB', part).group(1)) * 1024
        elif 'GB' in part:
            total += float(re.search(r'(\d+\.?\d*)GB', part).group(1))
    
    return pd.Series([total, ssd])

# Aplicamos a train y test
df[['total_storage_gb', 'ssd_storage_gb']] = df['memory'].apply(parse_storage)
test[['total_storage_gb', 'ssd_storage_gb']] = test['memory'].apply(parse_storage)

In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 912 entries, 755 to 229
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   company           912 non-null    object 
 1   product           912 non-null    object 
 2   typename          912 non-null    object 
 3   inches            912 non-null    float64
 4   screenresolution  912 non-null    object 
 5   cpu               912 non-null    object 
 6   ram               912 non-null    object 
 7   memory            912 non-null    object 
 8   gpu               912 non-null    object 
 9   opsys             912 non-null    object 
 10  weight            912 non-null    object 
 11  price_in_euros    912 non-null    float64
 12  ram_in_gb         912 non-null    float64
 13  weight_in_kg      912 non-null    float64
 14  res_y             912 non-null    float64
 15  total_storage_gb  912 non-null    float64
 16  ssd_storage_gb    912 non-null    float64
dtype

In [9]:
# One-hot encoding de las categóricas
df = pd.get_dummies(df, columns=["company", "typename", "opsys", "cpu", "gpu"], drop_first=True)
test = pd.get_dummies(test, columns=["company", "typename", "opsys", "cpu", "gpu"], drop_first=True)

-----------------------------------------------------------------------------------------------------------------

## Modelado

### 1. Definir X e y

In [10]:
# Selección de features
X = df.drop(columns=["price_in_euros", "product", "screenresolution", "ram", "memory", "weight"])
y = df["price_in_euros"]

### 2. Dividir X_train, X_test, y_train, y_test

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

### 3. Modelo

In [12]:
# Instanciamos el modelo
rf = RandomForestRegressor(random_state=42)

# Creamos el grid de hiperparámetros
param_grid = {
    'n_estimators': [300, 500, 750],
    'max_depth': [15, 20, 25],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2', 0.7]
}

# Grid Search con RMSE negativo (maximizamos la raiz negativa para minimizar el RMSE)
grid_search = GridSearchCV(estimator=rf,
                           param_grid=param_grid,
                           scoring="neg_root_mean_squared_error",
                           cv=5,
                           n_jobs=-1,
                           verbose=2)

# Entrenamos
grid_search.fit(X_train, y_train)

# Mostramos el mejor resultado
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor RMSE:", -grid_search.best_score_)

# Guardamos el mejor modelo
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 162 candidates, totalling 810 fits
Mejores hiperparámetros: {'max_depth': 25, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}
Mejor RMSE: 282.282105955267


In [13]:
# Aplicamos la misma selección de columnas que a X
X_kaggle_test = test.reindex(columns=X.columns, fill_value=0)

# Predecimos sobre ese conjunto
predictions_submit = best_model.predict(X_kaggle_test)

### 3. **¿Qué es lo que subirás a Kaggle?**

**Para subir a Kaggle la predicción esta tendrá que tener una forma específica.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [14]:
sample = pd.read_csv("./data/sample_submission.csv")

In [15]:
sample.head()

,laptop_ID,Price_in_euros
0,209,1949.1
1,1281,805.0
2,1168,1101.0
3,1231,1293.8
4,1020,1832.6


In [16]:
sample.shape

(391, 2)

### 4. Mete tus predicciones en un dataframe llamado ``submission``.

In [17]:
# Creamos el DataFrame de submission con la estructura correcta
submission = pd.DataFrame({
    "laptop_ID": test.index,
    "Price_in_euros": predictions_submit
})

In [18]:
submission.head()

,laptop_ID,Price_in_euros
0,209,1492.261591
1,1281,303.479749
2,1168,444.447571
3,1231,1005.198526
4,1020,853.031613


In [19]:
submission.shape

(391, 2)

### 5. Pásale el CHEQUEADOR para comprobar que efectivamente está listo para subir a Kaggle.

In [20]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                submission.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [21]:
chequeador(submission)

You're ready to submit!
